In [77]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from  sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import model_selection
import plotly.express as px

In [118]:
import rfpimp

In [119]:
df = pd.read_csv("../data/raw/bank-additional-full.csv", sep=";")

Link to Dataset: https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset
It is helpful to know that for all missing categorical values the string "unknown" is used.

In [120]:
df.shape

(41188, 21)

Drop duplicates.

In [121]:
df.drop_duplicates(inplace=True)

In [122]:
df.shape

(41176, 21)

In [123]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [124]:
list(df.columns)

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [125]:
df.rename(columns={"emp.var.rate":"emp_var_rate", "cons.price.idx":"cons_price_idx", "cons.conf.idx":"cons_conf_idx", "nr.employed":"nr_employed"}, inplace=True)

In [126]:
df[df.age == "unknown"]

/Users/Anti/.pyenv/versions/3.6.10/envs/data-science/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y


In [127]:
df.age = df.age.astype("uint8")

No unknown values in the age column.

In [128]:
df.job.value_counts()

admin.           10419
blue-collar       9253
technician        6739
services          3967
management        2924
retired           1718
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [129]:
df["job"].replace({"admin.":"admin"}, inplace=True)

I replaces "admin." with admin because I see no reason for the dot.

In [130]:
(df[df.job == "unknown"].shape[0] / df.shape[0]) * 100

0.8014377307169226

Dropping the rows with unknown jobs would lead to a loss of 0.8% of data which I think is acceptable.

In [131]:
df = df[df.job != "unknown"]

In [132]:
df.shape

(40846, 21)

Let's have a look on the marital column.

In [133]:
df.marital.value_counts()

married     24687
single      11490
divorced     4598
unknown        71
Name: marital, dtype: int64

In [134]:
(df[df.marital == "unknown"].shape[0] / df.shape[0]) * 100

0.17382363022082947

0.17% of the rows have an unknown marital status. I consider dropping them.

In [135]:
df = df[df.marital != "unknown"]

In [136]:
df.shape

(40775, 21)

Next I gonna have a look on the education column.

In [137]:
df.education.value_counts()

university.degree      12092
high.school             9461
basic.9y                6006
professional.course     5222
basic.4y                4118
basic.6y                2263
unknown                 1595
illiterate                18
Name: education, dtype: int64

1569 values unknown is too much for dropping the rows I think. So I think keeping them and later on using the unknown value as a column for one hot encoding would be appropriate.

Next the default column.

In [138]:
df.default.value_counts()

no         32337
unknown     8435
yes            3
Name: default, dtype: int64

A lot of missing data. I try using "unknown" as a column for one hot encoding again.

Column housing:

In [139]:
df.housing.value_counts()

yes        21371
no         18420
unknown      984
Name: housing, dtype: int64

I think i will try the same method as for default and education for all categorical columns unless in my comments is written somethin different.

Column loan:

In [140]:
df.loan.value_counts()

no         33608
yes         6183
unknown      984
Name: loan, dtype: int64

Column contact:

In [141]:
df.contact.value_counts()

cellular     25919
telephone    14856
Name: contact, dtype: int64

Cool! No missing data. :D

Column month:

In [142]:
df.month.value_counts()

may    13617
jul     7113
aug     6106
jun     5245
nov     4086
apr     2619
oct      706
sep      562
mar      541
dec      180
Name: month, dtype: int64

Column day_of_week

In [143]:
df.day_of_week.value_counts()

thu    8525
mon    8418
wed    8052
tue    8002
fri    7778
Name: day_of_week, dtype: int64

Column duration:

In [144]:
df.duration.value_counts()

90      169
85      168
136     167
73      163
124     162
       ... 
1201      1
1233      1
1265      1
2025      1
2015      1
Name: duration, Length: 1541, dtype: int64

In [145]:
df.duration = df.duration.astype("uint8")

Column campaign:

In [146]:
df.campaign.value_counts()

1     17444
2     10486
3      5294
4      2627
5      1586
6       966
7       622
8       394
9       277
10      222
11      176
12      124
13       89
14       69
17       57
16       50
15       49
18       31
20       30
19       26
21       23
22       17
23       16
24       15
27       11
29       10
25        8
26        8
28        8
30        7
31        7
35        5
33        4
32        4
34        3
40        2
42        2
43        2
37        1
39        1
41        1
56        1
Name: campaign, dtype: int64

Column pdays:

In [147]:
df.pdays.value_counts(ascending=True)

27         1
25         1
20         1
26         1
21         2
22         3
19         3
18         7
17         8
16        10
0         15
8         17
14        20
15        24
1         25
11        27
13        35
5         46
10        52
12        58
2         59
7         60
9         64
4        116
6        404
3        431
999    39285
Name: pdays, dtype: int64

In [148]:
df.pdays = df.pdays.astype("uint8")

Column previous:

In [149]:
df.previous.value_counts()

0    35205
1     4522
2      740
3      214
4       70
5       18
6        5
7        1
Name: previous, dtype: int64

In [150]:
df.previous = df.previous.astype("uint8")

Cloumn poutcome:

In [151]:
df.poutcome.value_counts()

nonexistent    35205
failure         4221
success         1349
Name: poutcome, dtype: int64

Column emp_var_rate

In [152]:
df.emp_var_rate.value_counts()

 1.4    16064
-1.8     9126
 1.1     7651
-0.1     3671
-2.9     1637
-3.4     1052
-1.7      763
-1.1      631
-3.0      170
-0.2       10
Name: emp_var_rate, dtype: int64